In [1]:
aois = ["Bab el-Mandeb Strait", "Cape of Good Hope", "Suez Canal"]
countries_of_interest = [
    "Egypt",
    "Yemen",
    "Djibouti",
    "Eritrea",
    "Saudi Arabia",
    "Jordan",
]
ISO_COUNTRIES = [818, 887, 262, 232, 682, 400]
START_DATE = "2023-01-01"

In [2]:
%reload_ext autoreload
%autoreload 2
import logging

import os
import sys
from os.path import join

import pandas as pd

import git

sys.path.append("red_sea_monitoring")

git_repo = git.Repo(os.getcwd(), search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")
sys.path.append(join(git_root, "src", "red_sea_monitoring"))
import visuals
import acled
# from red_sea_monitoring.acled import *

from datetime import date
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta

import os

logger = logging.getLogger()
logging.basicConfig(format="%(asctime)s %(message)s", level=logging.INFO)

from plotnine import *

# Armed Conflict Location Event Analysis

This section examines how the conflict fatalities in the countries of the red sea region have progressed since the crisis started in October 7th. The analysis is conducted using dat from ACLED. 

## Insights

To match the conflict analysis with the maritime trade trends as [previously seen on this web-book](https://datapartnership.org/red-sea-monitoring/notebooks/ports/README.html), we aggregated the ACLED data to show weekly trends near the ports of interest. 

### Visualizing conflict fatalities between January 1st 2023 and September 31st 2024

In [50]:
data = pd.DataFrame()
# Define the start date (January 1, 2023)
start_date = datetime(2023, 1, 1)

# Define the end date (today's date)
final_end_date = datetime.now()

# Loop through each 3-month period
current_date = start_date

while current_date < final_end_date:
    # Calculate the end of the 3-month period
    period_end = current_date + relativedelta(months=3) - timedelta(days=1)

    # If period_end exceeds the current date, adjust it to today
    if period_end > final_end_date:
        period_end = final_end_date

    # Format the start and end dates in ISO format (YYYY-MM-DD)
    start_date_str = current_date.date().isoformat()
    end_date_str = period_end.date().isoformat()

    # Call the ACLED API for the current 3-month period
    acled_data = acled.acled_api(
        email_address=os.environ.get("ACLED_EMAIL"),
        access_key=os.environ.get("ACLED_KEY"),
        country=ISO_COUNTRIES,
        start_date=start_date_str,
        end_date=end_date_str
    )

    data = pd.concat([data, acled_data])

    # Do something with the API response (e.g., store it, process it, etc.)
    print(f"Data from {start_date_str} to {end_date_str}: {acled_data.shape[0]}")

    # Move to the next 3-month period
    current_date = current_date + relativedelta(months=3)


c:\WBG\Anaconda3\envs\data-goods\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.acleddata.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings


Data from 2023-01-01 to 2023-03-31: 1827


c:\WBG\Anaconda3\envs\data-goods\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.acleddata.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings


Data from 2023-04-01 to 2023-06-30: 1242


c:\WBG\Anaconda3\envs\data-goods\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.acleddata.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings


...

c:\WBG\Anaconda3\envs\data-goods\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.acleddata.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings


Data from 2024-04-01 to 2024-06-30: 2336


c:\WBG\Anaconda3\envs\data-goods\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.acleddata.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings


Data from 2024-07-01 to 2024-09-30: 3290
Data from 2024-10-01 to 2024-10-03: 0


c:\WBG\Anaconda3\envs\data-goods\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.acleddata.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings


In [51]:
data["latitude"] = data["latitude"].astype("float64")
data["longitude"] = data["longitude"].astype("float64")
data["fatalities"] = data["fatalities"].astype("int")

In [52]:
data.drop_duplicates(inplace=True)

In [53]:
import geopandas as gpd
from shapely import Point


def convert_to_gdf(data):
    geometry = [Point(xy) for xy in zip(data["longitude"], data["latitude"])]
    gdf = gpd.GeoDataFrame(data, geometry=geometry, crs="EPSG:4326")

    return gdf

In [8]:
conflict_red_sea = data[
    (data["event_type"] != "Protests") & (data["location"] == "South Red Sea")
][
    [
        "country",
        "latitude",
        "longitude",
        "event_type",
        "sub_event_type",
        "location",
        "event_date",
        "fatalities",
    ]
]
conflict_red_sea = convert_to_gdf(conflict_red_sea)

conflict_red_sea = (
    conflict_red_sea.groupby(["country", "event_date", "location"])["fatalities"]
    .agg(["sum", "count"])
    .reset_index()
)
conflict_red_sea.rename(
    columns={"sum": "nrFatalities", "count": "nrEvents"}, inplace=True
)

conflict_red_sea['event_date'] = conflict_red_sea['event_date'].apply(lambda x: pd.to_datetime(x))

In [9]:
conflict_houthi = data[
    (data["event_type"] != "Protests") &((data['notes'].str.contains('Houthi')))
][
    [
        "country",
        "latitude",
        "longitude",
        "event_type",
        "sub_event_type",
        "location",
        "event_date",
        "fatalities","actor1", "actor2", "notes"
    ]
]
conflict_houthi = convert_to_gdf(conflict_houthi)
conflict_houthi = conflict_houthi[conflict_houthi['event_date']>='2023-10-01']
conflict_houthi['fatalities'] = conflict_houthi['fatalities'].astype(int)
conflict_houthi['fatalities']=conflict_houthi['fatalities'].fillna(0)
# conflict_red_sea = conflict_red_sea.groupby(['country', 'event_date'])['fatalities'].agg(['sum','count']).reset_index()
# conflict_red_sea.rename(columns = {"sum": "nrFatalities", "count": "nrEvents"}, inplace=True)

In [45]:
grouped_data = convert_to_gdf(
    data.groupby(["latitude", "longitude"])["fatalities"]
    .agg(["sum", "count"])
    .reset_index()
)
grouped_data.rename(
    columns={"sum": "nr_fatalities", "count": "nr_events"}, inplace=True
)

In [20]:
conflict_monthly = data[
    (data["event_type"] != "Protests")
][
    [
        "country",
        "latitude",
        "longitude",
        "event_type",
        "sub_event_type",
        "location",
        "event_date",
        "fatalities",
    ]
]
conflict_monthly['event_date'] = conflict_monthly['event_date'].apply(lambda x: pd.to_datetime(x))
conflict_monthly = (
    conflict_monthly.groupby(["country", pd.Grouper(key="event_date", freq='MS'), "latitude", "longitude"])["fatalities"]
    .agg(["sum", "count"])
    .reset_index()
)
conflict_monthly.rename(
    columns={"sum": "nrFatalities", "count": "nrEvents"}, inplace=True
)

conflict_monthly['event_date']= conflict_monthly['event_date'].astype('str')

In [3]:
events_dict = {datetime(2023, 10, 7):'First Attack on Gaza',
    datetime(2023, 11, 17):'First Attack on the Red Sea',
    
               #datetime(2022, 10,5): 'West Azerbaijan\nEarthquake',
               }

In [44]:
import folium
from folium.plugins import TimestampedGeoJson
import pandas as pd
import json


# Create the base map
m = folium.Map(location=[20.0, 38.5], zoom_start=5)

# Create a list to hold the geojson features
features = []

# Iterate over the DataFrame rows and create a geojson feature for each event
for _, row in conflict_monthly.iterrows():
    feature = {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [row['longitude'], row['latitude']],
        },
        'properties': {
            'time': row['event_date'],
            'popup': f"Fatalities: {row['nrFatalities']}<br>Events: {row['nrEvents']}",
            'icon': 'circle',
            'iconstyle': {
                'fillColor': 'red',
                'fillOpacity': 0.6,
                'stroke': 'false',
                'color':None,
                'radius': row['nrFatalities']/5  # Scale marker size by 'nr_events'
            }
        }
    }
    features.append(feature)

# Create the TimestampedGeoJson layer
timestamped_geojson = TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': features
    },
    period='P1M',  # One month between timestamps
    add_last_point=True,
    auto_play=False,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options='YYYY-MM-DD',
    time_slider_drag_update=True
)

# Add the TimestampedGeoJson to the map
timestamped_geojson.add_to(m)

# Show the map
m


In [54]:
grouped_data = grouped_data[grouped_data["nr_fatalities"] > 0]

In [60]:
m = grouped_data.explore(
    column="nr_fatalities",
    zoom_start=5.1,
    #marker_kwds={"radius": 'nr_events'},
    vmin=1,
    vmax=50,
    cmap="viridis",
)
m

In [110]:
from bokeh.plotting import show, output_notebook
import bokeh
from bokeh.core.validation.warnings import EMPTY_LAYOUT, MISSING_RENDERERS
from bokeh.models import Panel, Tabs

output_notebook()

bokeh.core.validation.silence(EMPTY_LAYOUT, True)
bokeh.core.validation.silence(MISSING_RENDERERS, True)

measure_names = {
    "nrEvents": "Number of Conflict Events",
    "nrFatalities": "Number of Fatalities",
}
measure_colors = {"nrEvents": "#4E79A7", "nrFatalities": "#F28E2B"}

show(
    visuals.get_bar_chart(
        conflict_red_sea,
        "Reported Attacks in the Red Sea Region",
        "All incidents were reported in the Yemen region. The sub event types that were identified in these attacks are Armed clash, Air/drone strike, Remote explosive/landmine/IED, \nShelling/artillery/missile attack, Arrests,Change to group/activity, Disrupted weapons use, Abduction/forced disappearance\nSource: ACLED",
        subtitle="",
        category=None,
        measure="nrEvents",
        color_code=measure_colors["nrEvents"],
        category_value="Yemen",
        events_dict=events_dict
    ),
    title=country.title(),
)

Loading BokehJS ...

**Reported attacks in the Red Sea region, particularly in Yemen, have markedly risen since the onset of the conflict in the Middle East.** 

The death toll since April has risen from 11 fatalities in the Red Sea region to 15. The conflict events have risen from 82 to 185. The previous peak seen in March 2024 has been replaced by a new peak in June 2024 which registered 34 conflict events. In the entire region, the number of Houthi-related deaths are 1754 and number of conflict events are 1264 since October 2023. 


---------------
**Updates from April 2024**

Since the start of the Red Sea Crisis on November 17, 2023, until April 11, 2024, 82 conflict events have occurred in the Red Sea region, resulting in 11 fatalities. The conflict events have increased, with March 2024 registering 27 conflict events, the highest so far. These events include anti-ship ballistic missiles launched by Houthi forces and interception and strike efforts by the US, UK, German, French, and Italian Navies. Overall, conflict events involving Houthi forces have resulted in 956 conflict events and 959 conflict-related fatalities across the MENA (Middle East and North Africa) region since the start of the conflict in Gaza.  

### Ports of Interest in the Red Sea Region

In [106]:
ports_red_sea = gpd.read_file(
    join(git_root, "data", "red_sea_ports.geojson"), driver="GeoJSON"
)

In [107]:
import folium

radius = 50 * 1000
ports_red_sea = ports_red_sea.to_crs(epsg=32633)
ports_red_sea["geometry"] = ports_red_sea["geometry"].apply(lambda x: x.buffer(radius))


folium.GeoJson(
    ports_red_sea,
    name="Ports",
    # Customize the style if needed
    style_function=lambda feature: {
        "color": "#CC5500",
        "weight": 2,
        "fillColor": "#CC5500",
    },
).add_to(m)

m

In [49]:
data["event_date"] = pd.to_datetime(data["event_date"])

gdf = convert_to_gdf(data)

In [50]:
conflict_by_country = (
    data.groupby(["country", pd.Grouper(key="event_date", freq="W")])["fatalities"]
    .agg(["sum", "count"])
    .reset_index()
)
conflict_by_country.rename(
    columns={"sum": "nrFatalities", "count": "nrEvents"}, inplace=True
)

In [51]:
conflict_by_country_event = (
    data.groupby(
        [
            "country",
            "event_type",
            "sub_event_type",
            pd.Grouper(key="event_date", freq="W"),
        ]
    )["fatalities"]
    .agg(["sum", "count"])
    .reset_index()
)
conflict_by_country_event.rename(
    columns={"sum": "nrFatalities", "count": "nrEvents"}, inplace=True
)

In [52]:
conflict_by_port = gpd.sjoin_nearest(
    ports_red_sea.to_crs(epsg=32633),
    gdf.to_crs(epsg=32633),
    max_distance=150 * 1000,
    distance_col="distance",
    how="right",
)

In [53]:
conflict_by_port = conflict_by_port[
    conflict_by_port["country_left"] == conflict_by_port["country_right"]
]
conflict_by_port = conflict_by_port[~(conflict_by_port["index_left"].isnull())]

In [54]:
conflict_by_port = conflict_by_port[
    conflict_by_port["country_left"].isin(["Yemen", "Jordan"])
]

In [55]:
conflict_by_port = (
    conflict_by_port.groupby(
        [
            "portid",
            "portname",
            "fullname",
            "event_type",
            pd.Grouper(key="event_date", freq="W"),
            "country_right",
        ]
    )["fatalities"]
    .agg(["sum", "count"])
    .reset_index()
)
conflict_by_port.rename(
    columns={"sum": "nrFatalities", "count": "nrEvents"}, inplace=True
)

In [151]:
from bokeh.plotting import show, output_notebook
import bokeh
from bokeh.core.validation.warnings import EMPTY_LAYOUT, MISSING_RENDERERS
from bokeh.models import Panel, Tabs, TabPanel

output_notebook()

bokeh.core.validation.silence(EMPTY_LAYOUT, True)
bokeh.core.validation.silence(MISSING_RENDERERS, True)

conflict_by_country = conflict_by_country.sort_values(by="country", ascending=False)

tabs = []
measure_names = {
    "nrEvents": "Number of Conflict Events",
    "nrFatalities": "Number of Fatalities",
}
measure_colors = {"nrEvents": "#4E79A7", "nrFatalities": "#F28E2B"}
# acled_adm0 = get_acled_by_admin(syria_adm2_crs, acled, columns = ['ADM2_EN', 'ADM1_EN'])
for country in list(conflict_by_country["country"].unique()):
    tabs.append(
        TabPanel(
            child=visuals.get_bar_chart(
                conflict_by_country,
                f"Weekly Conflict Event Trend in {country}",
                "Source: ACLED",
                subtitle="",
                category="country",
                measure="nrEvents",
                color_code=measure_colors["nrEvents"],
                category_value=country,
                events_dict=events_dict
            ),
            title=country.title(),
        )
    )

tabs = Tabs(tabs=tabs, sizing_mode="scale_both")
show(tabs, warn_on_missing_glyphs=False)

Loading BokehJS ...

In [97]:
from bokeh.plotting import show, output_notebook
import bokeh
from bokeh.core.validation.warnings import EMPTY_LAYOUT, MISSING_RENDERERS
from bokeh.models import Panel, Tabs

output_notebook()

bokeh.core.validation.silence(EMPTY_LAYOUT, True)
bokeh.core.validation.silence(MISSING_RENDERERS, True)

conflict_by_country = conflict_by_country.sort_values(by="country", ascending=False)

tabs = []
measure_names = {
    "nrEvents": "Number of Conflict Events",
    "nrFatalities": "Number of Fatalities",
}
measure_colors = {"nrEvents": "#4E79A7", "nrFatalities": "#F28E2B"}
# acled_adm0 = get_acled_by_admin(syria_adm2_crs, acled, columns = ['ADM2_EN', 'ADM1_EN'])
for country in list(conflict_by_country["country"].unique()):
    tabs.append(
        TabPanel(
            child=visuals.get_bar_chart(
                conflict_by_country,
                f"Weekly Conflict Fatality Trend in {country}",
                "Source: ACLED",
                subtitle="",
                category="country",
                measure="nrFatalities",
                color_code=measure_colors["nrFatalities"],
                category_value=country,
                events_dict=events_dict
            ),
            title=country.title(),
        )
    )

tabs = Tabs(tabs=tabs, sizing_mode="scale_both")
show(tabs, warn_on_missing_glyphs=False)

Loading BokehJS ...

In [59]:
port_names = (
    conflict_by_port[["portname", "fullname"]]
    .set_index("portname")
    .to_dict()["fullname"]
)

In [60]:
conflict_by_port.sort_values(by=["country_right", "event_date"], inplace=True)

### Weekly Conflict Events in the South Red Sea Region (excluding protests)

In [109]:
output_notebook()
from bokeh.core.validation.warnings import EMPTY_LAYOUT, MISSING_RENDERERS
from bokeh.models import Panel, Tabs

bokeh.core.validation.silence(EMPTY_LAYOUT, True)
bokeh.core.validation.silence(MISSING_RENDERERS, True)

conflict_by_port = conflict_by_port.sort_values(by="portname", ascending=True)
conflict_by_port_without_protest = conflict_by_port[
    conflict_by_port["event_type"] != "Protests"
]
event_types = list(conflict_by_port_without_protest["event_type"].unique())

tabs = []
measure_names = {
    "nrEvents": "Number of Conflict Events",
    "nrFatalities": "Number of Fatalities",
}
measure_colors = {"nrEvents": "#4E79A7", "nrFatalities": "#F28E2B"}
# acled_adm0 = get_acled_by_admin(syria_adm2_crs, acled, columns = ['ADM2_EN', 'ADM1_EN'])
for port in list(conflict_by_port_without_protest["portname"].unique()):
    tabs.append(
        TabPanel(
            child=visuals.get_stacked_bar_chart(
                conflict_by_port_without_protest[
                    conflict_by_port_without_protest["portname"] == port
                ],
                f"Weekly Conflict Event Trend in {port_names[port]}",
                "Source: ACLED",
                date_column="event_date",
                categories=event_types,
                measure="nrEvents",
                colors=[
                    "#4E79A7",
                    # "#F28E2B",
                    "#E15759",
                    "#76B7B2",
                    "#59A14F",
                    "#EDC948",
                ],
                events_dict=events_dict
            ),
            title=port.title(),
        )
    )

tabs = Tabs(tabs=tabs, sizing_mode="scale_both")
show(tabs, warn_on_missing_glyphs=False)

Loading BokehJS ...

#### Observations and Limitations



Loading BokehJS ...

In [83]:
conflict_by_country.to_csv(
    "../../data/conflict/conflict_by_country_2023-01-01_2024-10-01.csv"
)
conflict_by_port.to_csv(
    "../../data/conflict/conflict_by_port_2023-01-01_2024-10-01.csv"
)
data.to_csv("../../data/conflict/acled_raw_2023-01-01_2024-10-01.csv")